In [ ]:
!pip install transformers

In [ ]:
!pip install Torchtext==0.04

In [ ]:
!pip install datasets

In [ ]:
import torch
import pandas as pd
import transformers
from torchtext import data
from transformers import BertModel
import torch.nn as nn
import torch.optim as optim
from transformers import *
import math
import time
import torchtext
from datasets import load_dataset
from torch.utils.data import DataLoader
from sklearn.metrics import roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import numpy as np

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### BERT initial setup
Get default tokens with their IDs

In [ ]:
bert = BertModel.from_pretrained('bert-base-uncased')
bert_tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
cls = bert_tokenizer.cls_token
sep = bert_tokenizer.sep_token
pad = bert_tokenizer.pad_token
unk = bert_tokenizer.unk_token

cls_id = bert_tokenizer.cls_token_id
sep_id = bert_tokenizer.sep_token_id
pad_id = bert_tokenizer.pad_token_id
unk_id = bert_tokenizer.unk_token_id
print(cls_id,sep_id,pad_id,unk_id)

## Dataset setup -
Download and extracting dataset

In [ ]:
!wget https://nlp.stanford.edu/projects/snli/snli_1.0.zip
!unzip snli_1.0.zip

dataset_multinli_train = load_dataset('multi_nli',split='train') #split into train and val
dataset_multinli_test = load_dataset('multi_nli',split='validation_matched') #test

dataset_anli_train = load_dataset('anli',split='train_r2')
dataset_anli_dev = load_dataset('anli',split='dev_r2')
dataset_anli_test = load_dataset('anli',split='test_r2')


**Step 1 :** Loading dataset in dataframes

In [ ]:
#Read files
df_train_snli = pd.read_csv('snli_1.0/snli_1.0_train.txt', sep='\t')
df_dev_snli = pd.read_csv('snli_1.0/snli_1.0_dev.txt', sep='\t')
df_test_snli = pd.read_csv('snli_1.0/snli_1.0_test.txt', sep='\t')

df_train_snli = df_train_snli[['gold_label','sentence1','sentence2']].copy()
df_dev_snli = df_dev_snli[['gold_label','sentence1','sentence2']].copy()
df_test_snli = df_test_snli[['gold_label','sentence1','sentence2']].copy()

#Rename column names
df_train_snli.rename(columns = {'sentence1':'premise','sentence2':'hypothesis'}, inplace = True)
df_dev_snli.rename(columns = {'sentence1':'premise','sentence2':'hypothesis'}, inplace = True)
df_test_snli.rename(columns = {'sentence1':'premise','sentence2':'hypothesis'}, inplace = True)

# Remove rows with only one sentence
df_train_snli.dropna(inplace=True)

print(df_train_snli.head(2))


df_train_multinli = pd.DataFrame()
df_train_multinli['hypothesis'] = dataset_multinli_train['hypothesis']
df_train_multinli['premise'] = dataset_multinli_train['premise']
df_train_multinli['gold_label'] = dataset_multinli_train['label']
df_train_multinli.dropna(inplace=True)
df_train_multinli['gold_label'] = df_train_multinli['gold_label'].replace({0: 'neutral', 1: 'contradiction', 2: 'entailment'})

df_test_multinli = pd.DataFrame()
df_test_multinli['hypothesis'] = dataset_multinli_test['hypothesis']
df_test_multinli['premise'] = dataset_multinli_test['premise']
df_test_multinli['gold_label'] = dataset_multinli_test['label']
df_test_multinli.dropna(inplace=True)
df_test_multinli['gold_label'] = df_test_multinli['gold_label'].replace({0: 'neutral', 1: 'contradiction', 2: 'entailment'})


df_dev_multinli = df_train_multinli[len(df_train_multinli)-50000:]
df_train_multinli = df_train_multinli[:len(df_train_multinli)-50000]

print(df_train_multinli.head(2))
print(len(df_dev_multinli))
print(len(df_train_multinli))
print(len(df_test_multinli))

df_train_anli = pd.DataFrame()
df_train_anli['hypothesis'] = dataset_anli_train['hypothesis']
df_train_anli['premise'] = dataset_anli_train['premise']
df_train_anli['gold_label'] = dataset_anli_train['label']
df_train_anli.dropna(inplace=True)
df_train_anli['gold_label'] = df_train_anli['gold_label'].replace({0: 'neutral', 1: 'contradiction', 2: 'entailment'})

df_dev_anli = pd.DataFrame()
df_dev_anli['hypothesis'] = dataset_anli_dev['hypothesis']
df_dev_anli['premise'] = dataset_anli_dev['premise']
df_dev_anli['gold_label'] = dataset_anli_dev['label']
df_dev_anli.dropna(inplace=True)
df_dev_anli['gold_label'] = df_dev_anli['gold_label'].replace({0: 'neutral', 1: 'contradiction', 2: 'entailment'})

df_test_anli = pd.DataFrame()
df_test_anli['hypothesis'] = dataset_anli_test['hypothesis']
df_test_anli['premise'] = dataset_anli_test['premise']
df_test_anli['gold_label'] = dataset_anli_test['label']
df_test_anli.dropna(inplace=True)
df_test_anli['gold_label'] = df_test_anli['gold_label'].replace({0: 'neutral', 1: 'contradiction', 2: 'entailment'})


**Step 2 :** Preporcessing - limiting sentence length, adding token type and attention masks

In [8]:
#Helper functions for pre-processing
def limit_sentence_length(sentence):
  sentence = sentence.split(" ")
  sentence = sentence[:128]
  return " ".join(sentence)

def get_token_type(sentence,num):
  return [num]*len(sentence)


In [9]:
def preprocess_df(df_train,df_dev,df_test):
    #Limiting length
    df_train['premise_trim'] = df_train['premise'].apply(limit_sentence_length)
    df_train['hypothesis_trim'] = df_train['hypothesis'].apply(limit_sentence_length)

    df_dev['premise_trim'] = df_dev['premise'].apply(limit_sentence_length)
    df_dev['hypothesis_trim'] = df_dev['hypothesis'].apply(limit_sentence_length)

    df_test['premise_trim'] = df_test['premise'].apply(limit_sentence_length)
    df_test['hypothesis_trim'] = df_test['hypothesis'].apply(limit_sentence_length)

    #Adding starting and ending token
    df_train['premise_sos_eos'] = cls + ' ' + df_train['premise_trim'] + ' ' + sep + ' '
    df_train['hypothesis_sos_eos'] = df_train['hypothesis_trim'] + ' ' + sep

    df_dev['premise_sos_eos'] = cls + ' ' + df_dev['premise_trim'] + ' ' + sep + ' '
    df_dev['hypothesis_sos_eos'] = df_dev['hypothesis_trim'] + ' ' + sep

    df_test['premise_sos_eos'] = cls + ' ' + df_test['premise_trim'] + ' ' + sep + ' '
    df_test['hypothesis_sos_eos'] = df_test['hypothesis_trim'] + ' ' + sep

    # Tokenize
    df_train['premise_tokens'] = df_train['premise_sos_eos'].apply(bert_tokenizer.tokenize)
    df_train['hypothesis_tokens'] = df_train['hypothesis_sos_eos'].apply(bert_tokenizer.tokenize)

    df_dev['premise_tokens'] = df_dev['premise_sos_eos'].apply(bert_tokenizer.tokenize)
    df_dev['hypothesis_tokens'] = df_dev['hypothesis_sos_eos'].apply(bert_tokenizer.tokenize)

    df_test['premise_tokens'] = df_test['premise_sos_eos'].apply(bert_tokenizer.tokenize)
    df_test['hypothesis_tokens'] = df_test['hypothesis_sos_eos'].apply(bert_tokenizer.tokenize)

    #Added token types
    df_train['premise_token_type'] = df_train['premise_tokens'].apply(get_token_type,args={0})
    df_train['hypothesis_token_type'] = df_train['hypothesis_tokens'].apply(get_token_type,args={1})

    df_dev['premise_token_type'] = df_dev['premise_tokens'].apply(get_token_type,args={0})
    df_dev['hypothesis_token_type'] = df_dev['hypothesis_tokens'].apply(get_token_type,args={1})

    df_test['premise_token_type'] = df_test['premise_tokens'].apply(get_token_type,args={0})
    df_test['hypothesis_token_type'] = df_test['hypothesis_tokens'].apply(get_token_type,args={1})

    # Combine sentences
    df_train['combined_sentences'] = df_train['premise_tokens'] + df_train['hypothesis_tokens']
    df_dev['combined_sentences'] = df_dev['premise_tokens'] + df_dev['hypothesis_tokens']
    df_test['combined_sentences'] = df_test['premise_tokens'] + df_test['hypothesis_tokens']

    df_train['attention_mask'] = df_train['combined_sentences'].apply(get_token_type,args={1})
    df_dev['attention_mask'] = df_dev['combined_sentences'].apply(get_token_type,args={1})
    df_test['attention_mask'] = df_test['combined_sentences'].apply(get_token_type,args={1})

    df_train['token_type'] = df_train['premise_token_type'] + df_train['hypothesis_token_type']
    df_dev['token_type'] = df_dev['premise_token_type'] + df_dev['hypothesis_token_type']
    df_test['token_type'] = df_test['premise_token_type'] + df_test['hypothesis_token_type']

    # Get masks and token type of combined sentences
    df_train['combined_sentences'] = df_train['combined_sentences'].apply(lambda x:" ".join(x))
    df_dev['combined_sentences'] = df_dev['combined_sentences'].apply(lambda x:" ".join(x))
    df_test['combined_sentences'] = df_test['combined_sentences'].apply(lambda x:" ".join(x))

    df_train['attention_mask'] = df_train['attention_mask'].apply(lambda x : " ".join([str(y) for y in x]))
    df_dev['attention_mask'] = df_dev['attention_mask'].apply(lambda x : " ".join([str(y) for y in x]))
    df_test['attention_mask'] = df_test['attention_mask'].apply(lambda x : " ".join([str(y) for y in x]))

    df_train['token_type'] = df_train['token_type'].apply(lambda x : " ".join([str(y) for y in x]))
    df_dev['token_type'] = df_dev['token_type'].apply(lambda x : " ".join([str(y) for y in x]))
    df_test['token_type'] = df_test['token_type'].apply(lambda x : " ".join([str(y) for y in x]))

    # Remove temporary columns
    df_train = df_train[['gold_label', 'combined_sentences', 'attention_mask', 'token_type']]
    df_dev = df_dev[['gold_label', 'combined_sentences', 'attention_mask', 'token_type']]
    df_test = df_test[['gold_label', 'combined_sentences', 'attention_mask', 'token_type']]

    return df_train,df_dev,df_test



In [10]:
df_train_snli,df_dev_snli,df_test_snli = preprocess_df(df_train_snli,df_dev_snli,df_test_snli)
df_train_multinli,df_dev_multinli,df_test_multinli = preprocess_df(df_train_multinli,df_dev_multinli,df_test_multinli)
df_train_anli,df_dev_anli,df_test_anli = preprocess_df(df_train_anli,df_dev_anli,df_test_anli)

In [11]:
# Remove rows without label
df_train_snli = df_train_snli[df_train_snli['gold_label'] != '-'] 
df_dev_snli = df_dev_snli[df_dev_snli['gold_label'] != '-'] 
df_test_snli = df_test_snli[df_test_snli['gold_label'] != '-'] 

df_train_multinli = df_train_multinli[df_train_multinli['gold_label'] != '-'] 
df_dev_multinli = df_dev_multinli[df_dev_multinli['gold_label'] != '-'] 
df_test_multinli = df_test_multinli[df_test_multinli['gold_label'] != '-'] 

df_train_anli = df_train_anli[df_train_anli['gold_label'] != '-'] 
df_dev_anli = df_dev_anli[df_dev_anli['gold_label'] != '-'] 
df_test_anli = df_test_anli[df_test_anli['gold_label'] != '-'] 

In [12]:
#Save as csv
df_train_snli.to_csv('snli_1.0/snli_1.0_train.csv', index=False)
df_dev_snli.to_csv('snli_1.0/snli_1.0_dev.csv', index=False)
df_test_snli.to_csv('snli_1.0/snli_1.0_test.csv', index=False)

!mkdir multinli_1.0
df_train_multinli.to_csv('multinli_1.0/multinli_1.0_train.csv', index=False)
df_dev_multinli.to_csv('multinli_1.0/multinli_1.0_dev.csv', index=False)
df_test_multinli.to_csv('multinli_1.0/multinli_1.0_test.csv', index=False)

!mkdir anli_1.0
df_train_anli.to_csv('anli_1.0/anli_1.0_train.csv', index=False)
df_dev_anli.to_csv('anli_1.0/anli_1.0_dev.csv', index=False)
df_test_anli.to_csv('anli_1.0/anli_1.0_test.csv', index=False)

In [13]:
seq_snli = torchtext.data.Field(batch_first = True,use_vocab = False,tokenize = lambda x : x.strip().split(" ")[:254],preprocessing = bert_tokenizer.convert_tokens_to_ids,pad_token = pad_id,unk_token = unk_id)
labels_snli = torchtext.data.LabelField()
attention_mask_snli = torchtext.data.Field(batch_first = True,use_vocab = False,tokenize = lambda x : x.strip().split(" ")[:254],preprocessing = lambda x : [int(y) for y in x], pad_token = pad_id)
token_type_snli = torchtext.data.Field(batch_first = True,use_vocab = False,tokenize = lambda x : x.strip().split(" ")[:254],preprocessing = lambda x : [int(y) for y in x], pad_token = 1)

seq_multinli = torchtext.data.Field(batch_first = True,use_vocab = False,tokenize = lambda x : x.strip().split(" ")[:254],preprocessing = bert_tokenizer.convert_tokens_to_ids,pad_token = pad_id,unk_token = unk_id)
labels_multinli = torchtext.data.LabelField()
attention_mask_multinli = torchtext.data.Field(batch_first = True,use_vocab = False,tokenize = lambda x : x.strip().split(" ")[:254],preprocessing = lambda x : [int(y) for y in x], pad_token = pad_id)
token_type_multinli = torchtext.data.Field(batch_first = True,use_vocab = False,tokenize = lambda x : x.strip().split(" ")[:254],preprocessing = lambda x : [int(y) for y in x], pad_token = 1)

seq_anli = torchtext.data.Field(batch_first = True,use_vocab = False,tokenize = lambda x : x.strip().split(" ")[:254],preprocessing = bert_tokenizer.convert_tokens_to_ids,pad_token = pad_id,unk_token = unk_id)
labels_anli = torchtext.data.LabelField()
attention_mask_anli = torchtext.data.Field(batch_first = True,use_vocab = False,tokenize = lambda x : x.strip().split(" ")[:254],preprocessing = lambda x : [int(y) for y in x], pad_token = pad_id)
token_type_anli = torchtext.data.Field(batch_first = True,use_vocab = False,tokenize = lambda x : x.strip().split(" ")[:254],preprocessing = lambda x : [int(y) for y in x], pad_token = 1)


In [14]:
fields_snli = [('label', labels_snli), ('sequence', seq_snli), ('attention_mask', attention_mask_snli), ('token_type', token_type_snli)]

fields_multinli = [('label', labels_multinli), ('sequence', seq_multinli), ('attention_mask', attention_mask_multinli), ('token_type', token_type_multinli)]

fields_anli = [('label', labels_anli), ('sequence', seq_anli), ('attention_mask', attention_mask_anli), ('token_type', token_type_anli)]

train_data_snli, valid_data_snli, test_data_snli = torchtext.data.TabularDataset.splits(
                                        path = 'snli_1.0',
                                        train = 'snli_1.0_train.csv',
                                        validation = 'snli_1.0_dev.csv',
                                        test = 'snli_1.0_test.csv',
                                        format = 'csv',
                                        fields = fields_snli,
                                        skip_header = True)

train_data_multinli, valid_data_multinli, test_data_multinli = torchtext.data.TabularDataset.splits(
                                        path = 'multinli_1.0',
                                        train = 'multinli_1.0_train.csv',
                                        validation = 'multinli_1.0_dev.csv',
                                        test = 'multinli_1.0_test.csv',
                                        format = 'csv',
                                        fields = fields_multinli,
                                        skip_header = True)

train_data_anli, valid_data_anli, test_data_anli = torchtext.data.TabularDataset.splits(
                                        path = 'anli_1.0',
                                        train = 'anli_1.0_train.csv',
                                        validation = 'anli_1.0_dev.csv',
                                        test = 'anli_1.0_test.csv',
                                        format = 'csv',
                                        fields = fields_anli,
                                        skip_header = True)

labels_snli.build_vocab(train_data_snli)
labels_multinli.build_vocab(train_data_multinli)
labels_anli.build_vocab(train_data_anli)

In [15]:
#Parameters
batch_size_snli = 16
hidden_dim_snli = 512
output_dim_snli = len(labels_snli.vocab)



In [ ]:
batch_size_multinli = 16
hidden_dim_multinli = 512
output_dim_multinli = len(labels_multinli.vocab)



In [ ]:
batch_size_anli = 16
hidden_dim_anli = 512
output_dim_anli = len(labels_anli.vocab)

In [16]:
#Iterators
train_iterator_snli, valid_iterator_snli, test_iterator_snli = torchtext.data.BucketIterator.splits(
    (train_data_snli, valid_data_snli, test_data_snli), 
    batch_size = batch_size_snli,
    sort_key = lambda x: len(x.sequence),
    sort_within_batch = False, 
    device = device)



In [ ]:
train_iterator_multinli, valid_iterator_multinli, test_iterator_multinli = torchtext.data.BucketIterator.splits(
    (train_data_multinli, valid_data_multinli, test_data_multinli), 
    batch_size = batch_size_multinli,
    sort_key = lambda x: len(x.sequence),
    sort_within_batch = False, 
    device = device)



In [ ]:
train_iterator_anli, valid_iterator_anli, test_iterator_anli = torchtext.data.BucketIterator.splits(
    (train_data_anli, valid_data_anli, test_data_anli), 
    batch_size = batch_size_anli,
    sort_key = lambda x: len(x.sequence),
    sort_within_batch = False, 
    device = device)

In [17]:
#Model
class Bert_Model(nn.Module):
  def __init__(self,output_dim):
    super().__init__()
    self.bert = bert
    embedding_dim = bert.config.to_dict()['hidden_size']
    self.out = nn.Linear(embedding_dim, output_dim)
  def forward(self, seq, attention_mask, token_type):
    embeddings = self.bert(input_ids = seq, attention_mask = attention_mask, token_type_ids= token_type)[1]
    return self.out(embeddings)


In [ ]:
# Initialize models, optimizers, loss functions
#SNLI
bert_model_snli = Bert_Model(output_dim_snli).to(device)
optimizer_snli = AdamW(bert_model_snli.parameters(),lr=2e-5,eps=1e-6,correct_bias=False)
criterion_snli = nn.CrossEntropyLoss().to(device)


In [ ]:
#MultiNLI
bert_model_multinli = Bert_Model(output_dim_multinli).to(device)
optimizer_multinli = AdamW(bert_model_multinli.parameters(),lr=2e-5,eps=1e-6,correct_bias=False)
criterion_multinli = nn.CrossEntropyLoss().to(device)


In [ ]:
#ANLI
bert_model_anli = Bert_Model(output_dim_anli).to(device)
optimizer_anli = AdamW(bert_model_anli.parameters(),lr=2e-5,eps=1e-6,correct_bias=False)
criterion_anli = nn.CrossEntropyLoss().to(device)

In [19]:
#Reports
def get_reports(y_pred,y_true,roc_want=False):
  y_true = np.array([t.item() for t in y_true])
  y_pred = np.array([np.argmax(t.cpu()) for t in y_pred])

  accuracy = accuracy_score(y_true, y_pred)
  print("Accuracy for model:", accuracy)

  print('------- CLASSIFICATION REPORT --------')
  report = classification_report(y_true, y_pred)
  print(report)
  print('--------------------------------------')

  #Confusion matrix
  print('---------- CONFUSION MATRIX ----------')
  cmat = confusion_matrix(y_true, y_pred)
  print(cmat)
  print('--------------------------------------')


In [20]:
#Helper to calculate accuracy
def categorical_accuracy(preds, y):
    max_preds = preds.argmax(dim = 1, keepdim = True)
    correct = (max_preds.squeeze(1)==y).float()
    return correct.sum() / len(y)

In [21]:
#Train model
def train(bert_model,train_iterator,criterion,optimizer):
  epoch_loss = 0
  epoch_acc = 0
  bert_model.train()
  for batch in train_iterator:    
    optimizer.zero_grad()
    torch.cuda.empty_cache()
    predictions = bert_model(batch.sequence, batch.attention_mask, batch.token_type)
    loss = criterion(predictions, batch.label)
    # print(predictions.shape)
    acc = categorical_accuracy(predictions, batch.label)
    loss.backward()
    optimizer.step()
    # scheduler.step()
    epoch_loss += loss.item()
    epoch_acc += acc.item()
      
  return epoch_loss/len(train_iterator),epoch_acc/len(train_iterator)


In [22]:
#Evaluate model

def evaluate(bert_model,iterator,criterion):
  all_pred = []
  all_true = []
  epoch_loss = 0
  epoch_acc = 0
  bert_model.eval()
  with torch.no_grad():
    for batch in iterator:
      predictions = bert_model(batch.sequence, batch.attention_mask, batch.token_type)
      loss = criterion(predictions, batch.label)
      all_pred.extend(predictions)
      all_true.extend(batch.label)
      acc = categorical_accuracy(predictions, batch.label)
      
      epoch_loss += loss.item()
      epoch_acc += acc.item()
  
  return epoch_loss/len(iterator),epoch_acc/len(iterator),all_pred,all_true

In [23]:
def train_model(bert_model,train_iterator,valid_iterator,optimizer,criterion,dataset_name,epochs=2):
  
  all_pred = []
  all_true = []
  for epoch in range(epochs):
    start_time = time.time()
     
    train_loss, train_acc = train(bert_model,train_iterator,criterion,optimizer)
    valid_loss, valid_acc,predictions,true_labels = evaluate(bert_model,valid_iterator,criterion)
    
    end_time = time.time()
    torch.save(bert_model.state_dict(), 'bert_model_'+ dataset_name +'.pth')
    
    print('Epoch:',epoch+1,', Time taken:',str((end_time-start_time)/60))
    print('Train Loss:',train_loss,', Train Acc:',train_acc*100)
    print('Validation Loss:',valid_loss,', Validation Acc:',valid_acc*100)



In [25]:
# bert_model_snli = Bert_Model(output_dim_snli).to(device)
# bert_model_snli.load_state_dict(torch.load('bert_model.pth',map_location=torch.device('cpu')),)
# optimizer_snli = AdamW(bert_model_snli.parameters(),lr=2e-5,eps=1e-6,correct_bias=False)
# criterion_snli = nn.CrossEntropyLoss().to(device)

In [ ]:
train_model(bert_model_snli,train_iterator_snli,valid_iterator_snli,optimizer_snli,criterion_snli,'snli',epochs=2)

In [ ]:
train_model(bert_model_multinli,train_iterator_multinli,valid_iterator_multinli,optimizer_multinli,criterion_multinli,'multinli',epochs=4)

In [ ]:
train_model(bert_model_anli,train_iterator_anli,valid_iterator_anli,optimizer_anli,criterion_anli,'anli',epochs=4)

In [30]:
def get_test_accuracy(bert_model,test_iterator,criterion,dataset_name):
  bert_model.load_state_dict(torch.load('bert_model_'+ dataset_name +'.pth',map_location=torch.device('cpu')))
  test_loss, test_acc,all_pred_test,all_true_test = evaluate(bert_model,test_iterator,criterion)
  print('Test Loss:',test_loss,', Test Acc:',test_acc*100)
  get_reports(all_pred_test,all_true_test,False)

In [ ]:
get_test_accuracy(bert_model_snli,test_iterator_snli,criterion_snli,'snli')

In [ ]:
get_test_accuracy(bert_model_multinli,test_iterator_multinli,criterion_multinli,'multinli')

In [ ]:
get_test_accuracy(bert_model_anli,test_iterator_anli,criterion_anli,'anli')

In [ ]:
def predict_inference(premise, hypothesis, bert_model,labels):
    torch.cuda.empty_cache()
    bert_model.eval()    
    premise = cls + ' ' + premise + ' ' + sep
    hypothesis = hypothesis + ' ' + sep
    
    premise_tokens = bert_tokenizer.tokenize(premise)
    hypothesis_tokens = bert_tokenizer.tokenize(hypothesis)
   
    premise_token_type = get_token_type(premise_tokens,0)
    hypothesis_token_type = get_token_type(hypothesis_tokens,1)
    
    seq = premise_tokens + hypothesis_tokens
    seq = bert_tokenizer.convert_tokens_to_ids(seq)
    tokens_type = premise_token_type + hypothesis_token_type
    attention_mask = get_token_type(seq,1)
    
    seq = torch.LongTensor(seq).unsqueeze(0).to(device)
    tokens_type = torch.LongTensor(tokens_type).unsqueeze(0).to(device)
    attention_mask = torch.LongTensor(attention_mask).unsqueeze(0).to(device)
    
    prediction = bert_model(seq, attention_mask, tokens_type)
    prediction = prediction.argmax(dim=-1).item()
    
    return labels.vocab.itos[prediction]

In [ ]:
premise = 'a man sitting on a green bench.'
hypothesis = 'a woman sitting on a green bench.'

print(predict_inference(premise, hypothesis,bert_model_snli,labels_snli))
print(predict_inference(premise, hypothesis,bert_model_snli,labels_multinli))
print(predict_inference(premise, hypothesis,bert_model_snli,labels_anli))

contradiction
neutral


In [ ]:
premise = 'A black race car starts up in front of a crowd of people.'
hypothesis = 'A man is driving down a lonely road.'

print(predict_inference(premise, hypothesis,bert_model_snli,labels_snli))
print(predict_inference(premise, hypothesis,bert_model_snli,labels_multinli))
print(predict_inference(premise, hypothesis,bert_model_snli,labels_anli))

contradiction
neutral


In [ ]:
premise = 'A smiling costumed woman is holding an umbrella.'
hypothesis = 'A happy woman in a fairy costume holds an umbrella.'

print(predict_inference(premise, hypothesis,bert_model_snli,labels_snli))
print(predict_inference(premise, hypothesis,bert_model_snli,labels_multinli))
print(predict_inference(premise, hypothesis,bert_model_snli,labels_anli))

contradiction
neutral


In [ ]:
premise = 'A soccer game with multiple males playing.'
hypothesis = 'Some men are playing a sport.'

print(predict_inference(premise, hypothesis,bert_model_snli,labels_snli))
print(predict_inference(premise, hypothesis,bert_model_snli,labels_multinli))
print(predict_inference(premise, hypothesis,bert_model_snli,labels_anli))

entailment
entailment
